<a href="https://colab.research.google.com/github/haidour18/Benchemarking-Recommender-systems-/blob/main/Benchmarking__Etat_Art.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Install dependecies***

In [ ]:
!pip install kaggle
!pip install -q keras
!pip install numpy
!pip install sklearn
!pip install scipy
!pip install surprise
!pip install apyori

#***Kaggle Configuration***



In [ ]:
from google.colab import files 
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#change permissions 
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d zygmunt/goodbooks-10k

In [ ]:
# Dézippier le .zip obtenu
from zipfile import ZipFile
file_name = 'goodbooks-10k.zip'
with ZipFile (file_name ,'r') as zip : 
  zip.extractall()
  print('Done')

 
#***Data Preprocessing***



In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, glob , csv
import time, joblib
import pandas as pd
#from sklearn.model_selection import train_test_split
import surprise as sp 
from surprise.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import seaborn as sns

from surprise import KNNBasic,accuracy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from apyori import apriori
from IPython.display import display
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics.pairwise import cosine_similarity as cos
from sklearn.metrics import precision_score
import warnings; warnings.simplefilter('ignore')
from surprise import accuracy

In [ ]:
#Eliminating unneunnecessary columns from book.csv
input_file= '/content/books.csv'
output_file = '/content/books_output.csv'
cols_to_remove = [0,2,3,5,6,8,10,11,12,13,14,15,16,17,18,19,20,21,22] # Column indexes to be removed (starts at 0)
cols_to_remove = sorted(cols_to_remove, reverse=True) # Reverse so we remove from the end first

row_count = 0 # Current amount of rows processed

with open(input_file, "r") as source:
    reader = csv.reader(source)
    with open(output_file, "w", newline='') as result:
        writer = csv.writer(result)
        for row in reader:
            row_count += 1
            print('\r{0}'.format(row_count), end='') # Print rows processed
            for col_index in cols_to_remove:
                del row[col_index]
            writer.writerow(row)


In [ ]:
#Avanat d'exéuter cette section de code , il faut supprimer books.csv
#Jointure entre tous les fichiers csv dans un ficher csv nommé "dataset.csv"
path = "/content/"

all_files = glob.glob(os.path.join(path, "*.csv"))
df_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged   = pd.concat(df_file, ignore_index=True)
df_merged.to_csv( "dataset_genere.csv")

In [ ]:
dataset_complet=pd.read_csv("dataset_genere.csv")
#dataset_genere.tail(20)
del dataset_complet ['Unnamed: 0']

In [ ]:
dataset_complet

In [ ]:
#We can need it later 
#dataset1 =pd.read_csv('book_tags.csv')
dataset2= pd.read_csv('books_output.csv')
dataset3=pd.read_csv('ratings.csv')
#dataset4=pd.read_csv('tags.csv')
#dataset5=pd.read_csv('to_read.csv')
train=pd.merge(dataset2,dataset3)
train.head()

In [ ]:
# Splitting the dataset into the Training set and Test set
#train, test = train_test_split(dataset_complet, test_size=0.2)

In [ ]:
userRatings = train.pivot_table(index=['user_id'],columns=['original_title'],values='rating')

In [ ]:
userRatings=userRatings.dropna(thresh=10,axis=1).fillna(0)
#Here we are removing the movies having less than 10 user ratings and replacing NaN values with 0.
userRatings.head()

In [ ]:
ratings=dataset3[['user_id','book_id','rating']]
ratings = ratings.iloc[:20000,:]
reader = sp.Reader(rating_scale=(1,5)) 
result_dataset1 = sp.Dataset.load_from_df(ratings, reader)
train1,test1 = train_test_split(result_dataset1,test_size=0.2)

In [ ]:
user_model = sp.KNNBasic(k=40,sim_options={'name': 'pearson','user_based': True})
user_model.fit(train1)
preds = user_model.test(test1)
accuracy.rmse(preds,verbose=True)
accuracy.mae(preds,verbose=True)
accuracy.mse(preds,verbose=True)


In [ ]:
user_model = sp.KNNBasic(k=40,sim_options={'name': 'cosine','user_based': True})
user_model.fit(train1)
preds = user_model.test(test1)
accuracy.rmse(preds,verbose=True)
accuracy.mae(preds,verbose=True)
accuracy.mse(preds,verbose=True)

In [ ]:
item_model = sp.KNNBasic(k=40,sim_options={'name': 'pearson','user_based': False})
item_model.fit(train1)
preds = item_model.test(test1)
accuracy.rmse(preds,verbose=True)
accuracy.mae(preds,verbose=True)
accuracy.mse(preds,verbose=True)

In [ ]:
item_model = sp.KNNBasic(k=40,sim_options={'name': 'cosine','user_based': False})
item_model.fit(train1)
preds = item_model.test(test1)
accuracy.rmse(preds,verbose=True)
accuracy.mae(preds,verbose=True)
accuracy.mse(preds,verbose=True)

**This part isn't for execution ( just a try for the moment)**


In [ ]:
# Prevent errors with NMF
dataset3['rating'] = dataset3['rating'].where(dataset3['rating'] != 0, 1e-9)

testset, trainset = dataset3.loc[:180000-1 ], dataset3.iloc[180000:] 
print('Training set columns  :', trainset.shape)
print('Validation set columns:', testset.shape)
x_val, y_val = testset.drop('rating', axis=1), testset['rating'].values.reshape(-1,1)

In [ ]:
models = {
    'SVD': {'clf': sp.SVD(random_state=0)},
    'knn' : {'clf' :KNeighborsClassifier(n_neighbors=4)},
    'kmeans' : {'clf' : KMeans(n_clusters=4)},
    #'association_rules': {'clf' :apriori(dataset???,min_support=0.0045, min_confidence=0.2, min_lift=3, min_length=2)}
}

In [ ]:
metrics_list = []

def predict(row):
    return bench_model.predict(row['user_id'], row['book_id']).est

In [ ]:
reader = sp.Reader(rating_scale=(0, 5))
donnees= sp.Dataset.load_from_df(trainset, reader)
trainsett = donnees.build_full_trainset()
full_dataset = sp.Dataset.load_from_df(dataset3, reader)
full_trainset = full_dataset.build_full_trainset()

In [ ]:
for name, model in models.items():
    print('Fitting', name, 'model...', end='')
    
    bench_model = model['clf']
    time_start = time.time()
    bench_model.fit(trainsett)
    model['fit_time'] = time.time() - time_start

    print(' completed in', model['fit_time'], 'seconds.')

    print('Predicting ratings using', name, 'model...', end='')
    
    time_start = time.time()
    model['predictions'] = x_val.apply(predict, axis=1)
    model['predict_time'] = time.time() - time_start

    print(' completed in', model['predict_time'], 'seconds.')

    pred_mse = mse(y_val, model['predictions'])
    pred_mae = mae(y_val, model['predictions'])

    metrics_list.append({
        'Model': name,
        'Fit time (seconds)': model['fit_time'],
        'Predict time (seconds)': model['predict_time'],
        'MAE': pred_mae,
        'MSE': pred_mse,
        'RMSE': np.sqrt(pred_mse)
    })

**Content Based Recommender Systems **







In [ ]:
#Loading the dataset ##
dataset_TF_IDF=pd.read_csv("books.csv")
#some visualisation 
dataset_TF_IDF.head(19)


dataset_TF_IDF.columns



In [ ]:
#Preprocessing 
books_title = dataset_TF_IDF[['book_id', 'title']]
books_title.head()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1,2), min_df = 0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(books_title['title'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim